## tf.Variable

每次重新定义新的变量

In [1]:
import tensorflow as tf

In [2]:
a = tf.Variable([1.0], name='a')
b = tf.Variable([1.0], name='b')
c = tf.Variable([2.0], name='a')

In [3]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [4]:
sess.run(a)

array([1.], dtype=float32)

In [5]:
sess.run(c)

array([2.], dtype=float32)

In [6]:
c

<tf.Variable 'a_1:0' shape=(1,) dtype=float32_ref>

In [7]:
a

<tf.Variable 'a:0' shape=(1,) dtype=float32_ref>

In [8]:
sess.close()

## tf.get_variable

这个函数要求你指定变量的名称，这个名称将作为副本用于访问相同的变量，以及在检查点和导出模型时命名次变量的值。tf.get_variable还允许你重复使用先前创建的同名变量，从而轻松定义复用层的模型。

而直接通过tf.Variable来创建变量的话无论什么时候都会创建新的变量。

In [17]:
my_variable = tf.get_variable("my_variable", [1, 2, 3])

In [18]:
my_variable

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=float32_ref>

In [21]:
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], 
                                  dtype=tf.int32, initializer=tf.zeros_initializer)
other_variable = tf.get_variable("other_variable", 
                                 dtype=tf.int32, initializer=tf.constant([23, 42]))

In [22]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [23]:
sess.run(my_variable)

array([[[ 0.2586515 ,  1.0567503 , -0.15975052],
        [ 0.8708054 ,  0.20676029,  0.22349739]]], dtype=float32)

In [25]:
sess.close()

In [27]:
var1 = tf.get_variable("var", [1, 2, 3])

In [28]:
var2 = tf.get_variable("var", [1, 2, 3])

ValueError: Variable var already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1718, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3392, in create_op
    op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


## 共享变量

在构造一些网络的时候，可能会遇到一个层多次利用或多个输入使用同一个层的情况，这种时候就需要重复利用同一套权重变量，不然就无法达到预期的效果，这个时候就可以通过变量作用域tf.variable_scope()和tf.get_variable配合来实现

In [30]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
                                initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
                                initializer=tf.constant_initializer(0.0))
    print("conv2d")
    #conv = tf.nn.conv2d(input, weights,
    #                            strides=[1, 1, 1, 1], padding='SAME')
    #return tf.nn.relu(conv + biases)

在我们的真实模型中，有两个输入需要使用同一个卷积，于是你可能会想这么做：

In [31]:
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])

conv2d


第一次调用函数的时候就已经创建了weights和biases变量，第二次调用的时候变量名称已经存在，所以就无法再使用这两个名称了

将tf.get_variable改成 tf.Variable

In [34]:
def conv_relu2(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.Variable(kernel_shape, name = "weights")
    # Create variable named "biases".
    biases = tf.Variable(bias_shape, name = "biases")
    print("conv2d")
    print(weights, biases)
    #conv = tf.nn.conv2d(input, weights,
    #                            strides=[1, 1, 1, 1], padding='SAME')
    #return tf.nn.relu(conv + biases)

In [35]:
x = conv_relu2(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])

conv2d
<tf.Variable 'weights_1:0' shape=(4,) dtype=int32_ref> <tf.Variable 'biases_1:0' shape=(1,) dtype=int32_ref>


In [36]:
xx = conv_relu2(x, kernel_shape=[5, 5, 32, 32], bias_shape = [32])

conv2d
<tf.Variable 'weights_2:0' shape=(4,) dtype=int32_ref> <tf.Variable 'biases_2:0' shape=(1,) dtype=int32_ref>


两次调用不同的变量

如果要复用与第一次想通的变量，就需要使用变量作用域，并且声明重复使用变量，像下面这样操作：（设置reuse=True）

In [37]:
with tf.variable_scope("model"):
    output1 = tf.get_variable("my_variable", [1, 2, 3])
with tf.variable_scope("model", reuse=True):
    output2 = tf.get_variable("my_variable", [1, 2, 3])

In [38]:
output1

<tf.Variable 'model/my_variable:0' shape=(1, 2, 3) dtype=float32_ref>

In [39]:
output2

<tf.Variable 'model/my_variable:0' shape=(1, 2, 3) dtype=float32_ref>

或者：(设置scope.reuse_variables())

In [41]:
with tf.variable_scope("model2") as scope:
    output1 = tf.get_variable("my_variable", [1, 2, 3])
    scope.reuse_variables()
    output2 = tf.get_variable("my_variable", [1, 2, 3])

In [42]:
output1

<tf.Variable 'model2/my_variable:0' shape=(1, 2, 3) dtype=float32_ref>

In [43]:
output2

<tf.Variable 'model2/my_variable:0' shape=(1, 2, 3) dtype=float32_ref>

修改 conv_relu3

In [63]:
def conv_relu3(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    with tf.variable_scope("conv1"):  # 不使用reuse=True
        weights = tf.Variable(kernel_shape, name = "weights")
        # Create variable named "biases".
        biases = tf.Variable(bias_shape, name = "biases")
        
    print("conv2d")
    print(weights, biases)
    #conv = tf.nn.conv2d(input, weights,
    #                            strides=[1, 1, 1, 1], padding='SAME')
    #return tf.nn.relu(conv + biases)

In [64]:
conv_relu3(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])

conv2d
<tf.Variable 'conv1_8/weights:0' shape=(4,) dtype=int32_ref> <tf.Variable 'conv1_8/biases:0' shape=(1,) dtype=int32_ref>


In [65]:
conv_relu3(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])

conv2d
<tf.Variable 'conv1_9/weights:0' shape=(4,) dtype=int32_ref> <tf.Variable 'conv1_9/biases:0' shape=(1,) dtype=int32_ref>


## 总结

名称作用域tf.name_scope并不会对tf.get_variable产生影响，只会对tf.Variable之类的其他的命名操作增加一个名称范围前缀。

In [66]:
with tf.variable_scope("conv1"):
    a = tf.Variable([1.0], name='a')
with tf.variable_scope("conv2"):
    b = tf.Variable([1.0], name='a')
print(a)
print(b)

<tf.Variable 'conv1_10/a:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'conv2/a:0' shape=(1,) dtype=float32_ref>


In [67]:
with tf.variable_scope("conv1"):
    a = tf.get_variable('a', [1])
with tf.variable_scope("conv2"):
    b = tf.get_variable('a', [1])
print(a)
print(b)

<tf.Variable 'conv1/a:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'conv2/a_1:0' shape=(1,) dtype=float32_ref>


下面语句将会报错

In [70]:
with tf.variable_scope("conv1", reuse=True):
    a = tf.get_variable('a', [1])
with tf.variable_scope("conv1", reuse=True):
    b = tf.get_variable('a', [1])
print(a)
print(b)

<tf.Variable 'conv1/a:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'conv1/a:0' shape=(1,) dtype=float32_ref>


In [71]:
with tf.name_scope("conv1"):
    a = tf.get_variable('a', [1])
print(a)  #输出 (可以看出名称域对它没有影响)

<tf.Variable 'a_4:0' shape=(1,) dtype=float32_ref>


[深入理解TensorFlow变量](https://www.jianshu.com/p/c69f25fcc4a4)

## tensorflow中的两种作用域

name_scope和variable scope,之所以会出现这两种类型的scope，主要是后者（variable scope）为了实现tensorflow中的变量共享机制：即为了使得在代码的任何部分可以使用某一个已经创建的变量，TF引入了变量共享机制，使得可以轻松的共享变量，而不用传一个变量的引用。

- 命名域(name scope)：通过tf.name_scope()来实现；
- 变量域（variable scope）：通过tf.variable_scope()来实现；可以通过设置reuse 标志以及初始化方式来影响域下的变量。 

这两种作用域都会给tf.Variable()创建的变量加上词头，而tf.name_scope对tf.get_variable()创建的变量没有词头影响，


In [72]:
import tensorflow as tf

with tf.name_scope('cltdevelop'):
    var_1 = tf.Variable(initial_value=[0], name='var_1')
    var_2 = tf.get_variable(name='var_2', shape=[1, ])
with tf.variable_scope('aaa'):
    var_3 = tf.Variable(initial_value=[0], name='var_3')
    var_4 = tf.get_variable(name='var_4', shape=[1, ])


In [73]:
print(var_1.name)
print(var_2.name)
print(var_3.name)
print(var_4.name)

cltdevelop/var_1:0
var_2:0
aaa/var_3:0
aaa/var_4:0
